# Analyze DX cluster spots (Jupyter)

This notebook loads the `dx_spots` and `callsigns` tables from the repository SQLite database and provides interactive exploration cells: dataset summary, top-N lists, and plots.

Place this notebook in the `homework1` folder next to `dxcluster.db` (already done here). If you need plotting/analysis packages, install them first (see the next cell).

In [ ]:
# If you don't have the required packages installed you can run this cell once:
# Uncomment and run the line below in the notebook environment (it uses pip):
# !pip install -r ./requirements.txt

# Import libraries
import sqlite3
import os
try:
    import pandas as pd
except Exception as e:
    raise RuntimeError('Pandas is required for this notebook. Install with pip install pandas') from e

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set(style='whitegrid')
except Exception:
    plt = None

print('Libraries imported. Matplotlib available:', plt is not None)

In [ ]:
# Path to the sqlite DB. The notebook is in the same folder so the default filename should work.
DB_PATH = 'dxcluster.db'

if not os.path.exists(DB_PATH):
    raise FileNotFoundError(f'Could not find the database at {DB_PATH}. Make sure the file exists in this folder.')

# Load tables into pandas DataFrames
conn = sqlite3.connect(DB_PATH)
df_spots = pd.read_sql_query('SELECT * FROM dx_spots', conn, parse_dates=['timestamp'])
df_calls = pd.read_sql_query('SELECT * FROM callsigns', conn, parse_dates=['first_seen','last_seen'])
conn.close()

markdown
intro-1
markdown
# Analyze DX cluster spots (Jupyter)

This notebook loads the `dx_spots` and `callsigns` tables from the repository SQLite database and provides interactive exploration cells: dataset summary, top-N lists, and plots.

Place this notebook in the `homework1` folder next to `dxcluster.db`. If you need plotting/analysis packages, install them first (see the next cell).
code
imports-1
python

# If you don't have the required packages installed you can run this cell once:
# Uncomment and run the line below in the notebook environment (it uses pip):
# !pip install -r ./requirements.txt

# Import libraries
import sqlite3
import os
try:
    import pandas as pd
except Exception as e:
    raise RuntimeError('Pandas is required for this notebook. Install with pip install pandas') from e

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set(style='whitegrid')
except Exception:
    plt = None

print('Libraries imported. Matplotlib available:', plt is not None)
code
load-1
python

# Path to the sqlite DB. The notebook is in the same folder so the default filename should work.
DB_PATH = 'dxcluster.db'

if not os.path.exists(DB_PATH):
    raise FileNotFoundError(f'Could not find the database at {DB_PATH}. Make sure the file exists in this folder.')

# Load tables into pandas DataFrames
conn = sqlite3.connect(DB_PATH)
df_spots = pd.read_sql_query('SELECT * FROM dx_spots', conn, parse_dates=['timestamp'])
df_calls = pd.read_sql_query('SELECT * FROM callsigns', conn, parse_dates=['first_seen','last_seen'])
conn.close()

# Quick preview
df_spots.head()
code
summary-1
python

# Basic dataset summary
total_spots = len(df_spots)
unique_dx = df_spots['dx_call'].nunique()
unique_spotters = df_spots['spotter_call'].nunique()
times = pd.to_datetime(df_spots['timestamp'])
first = times.min()
last = times.max()
days = (last - first).days or 1
spots_per_day = total_spots / (days if days else 1)

print('Total spots:', total_spots)
print('Unique DX calls seen:', unique_dx)
print('Unique spotter calls:', unique_spotters)
print('Time range:', first, '->', last, f'({days} days)')
print(f'Average spots per day: {spots_per_day:.2f}')
code
topn-1
python

# Top-N lists (adjust n as needed)
def top_n(df, col, n=10):
    return df[col].value_counts().head(n)

n = 10
print('Top DX calls:')
display(top_n(df_spots, 'dx_call', n))

print('






























































































































































543.xpythonpython3pythonPython 3If you'd like, I can add an interactive folium map cell or a weekday/hour heatmap next.- Export aggregated CSVs for sharing or further analysis.- Use geolocation of grid squares to overlay activity on interactive maps (folium or kepler.gl).- Add weekday/weekend comparison and seasonal analysis.- Heatmap shows hour vs band activity; consider normalizing per-band to see relative activity.## Next steps and exploration ideasmarkdownnext-1markdown        plt.show()        plt.ylabel('Latitude')        plt.xlabel('Longitude')        plt.title('Grid-square spot density (approx center)')        plt.colorbar(label='Spot count')        plt.scatter(agg['lon'], agg['lat'], s=agg['count']*10, c=agg['count'], cmap='Reds', alpha=0.7, edgecolor='k')        plt.figure(figsize=(12,6))    except Exception:        plt.show()        ax.set_ylabel('Latitude')        ax.set_xlabel('Longitude')        ax.set_title('Grid-square spot density (approx center)')        plt.colorbar(sc, ax=ax, label='Spot count')        sc = ax.scatter(agg['lon'], agg['lat'], s=agg['count']*10, c=agg['count'], cmap='Reds', alpha=0.7, edgecolor='k')        world.plot(ax=ax, color='lightgray', edgecolor='white')        fig, ax = plt.subplots(figsize=(12,6))        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))        import geopandas as gpd    try:    # Plot positions; try geopandas world map if available, otherwise plain scatter    display(agg.head(20))    agg = agg.sort_values('count', ascending=False)    agg = df_grid.groupby('grid').agg(count=('id','size'), lat=('lat','mean'), lon=('lon','mean')).reset_index()    # Aggregate by grid to get counts    df_grid['lon'] = df_grid['latlon'].apply(lambda x: x[1])    df_grid['lat'] = df_grid['latlon'].apply(lambda x: x[0])    df_grid = df_grid.dropna(subset=['latlon']).copy()    df_grid['latlon'] = df_grid['grid'].apply(maiden_to_latlon)    df_grid['grid'] = df_grid['grid_square'].str.strip()else:    print('No grid_square data found in dataset.')if df_grid.empty:df_grid = df_spots.dropna(subset=['grid_square']).copy()# Extract grid squares and map to lat/lon        return None    except Exception:        return (lat, lon)            lat += 0.5            lon += 1.0            # center of 4-char square        else:            lat += (2.5/60.0) / 2            lon += (5.0/60.0) / 2            # center of subsquare            lat += (ord(g[5]) - A) * (2.5/60.0)            lon += (ord(g[4]) - A) * (5.0/60.0)        if len(g) >= 6:        lat += int(g[3]) * 1        lon += int(g[2]) * 2        lat = (ord(g[1]) - A) * 10 - 90        lon = (ord(g[0]) - A) * 20 - 180        A = ord('A')    try:        return None    if len(g) < 4:    g = grid.strip().upper()        return None    if not isinstance(grid, str):    # Accept 4 or 6-char grids; return (lat, lon) center of the squaredef maiden_to_latlon(grid):# Grid-square mapping: convert Maidenhead grid to lat/lon and plotpythongridsq-1code    plt.show()    plt.tight_layout()    plt.xlabel('Band')    plt.ylabel('Hour (UTC)')    plt.title('Heatmap: Hour (UTC) vs Band (activity count)')    sns.heatmap(pivot, cmap='viridis', linewidths=0.5, linecolor='gray')    plt.figure(figsize=(12,6))    pivot = pivot.reindex(range(24), fill_value=0)    # Reindex hours to 0-23 to show empty hours    pivot = df_spots.pivot_table(index='hour', columns='band', values='id', aggfunc='count', fill_value=0)    # Pivot table: rows=hour, cols=band, values=count    df_spots['hour'] = pd.to_datetime(df_spots['timestamp']).dt.hourelse:    print('Matplotlib/seaborn not available; install them to generate heatmap.')if plt is None:# Heatmap: hour (UTC) vs band activitypythonheatmap-1code    plt.show()    plt.tight_layout()    plt.xlabel('Spots')    plt.title('Top DX Calls')    sns.barplot(y=top_calls.index, x=top_calls.values, orient='h')    plt.figure(figsize=(8,6))    top_calls = df_spots['dx_call'].value_counts().head(20)    # Top DX calls    plt.show()    plt.xticks(rotation=45)    plt.xlabel('Band')    plt.ylabel('Count')    plt.title('Band distribution')    sns.barplot(x=band_counts.index, y=band_counts.values)    plt.figure(figsize=(8,5))    band_counts = df_spots['band'].value_counts(dropna=True)    # Band distribution    plt.show()    plt.ylabel('Count')    plt.title('Spots per day')    per_day.plot(kind='line')    plt.figure(figsize=(10,4))    per_day = df_spots.groupby('date').size()    df_spots['date'] = pd.to_datetime(df_spots['timestamp']).dt.date    # Spots per day    %matplotlib inlineelse:    print('Matplotlib/seaborn not available. Install them to enable plotting.')if plt is None:# Simple plots (inline). Requires matplotlib and seaborn.pythonplots-1codedisplay(busiest)print('
Top hours (UTC):')busiest = hours.value_counts().head(10)hours = pd.to_datetime(df_spots['timestamp']).dt.hour# Busiest UTC hourdisplay(df_spots['mode'].value_counts(dropna=True).head(20))print('
Mode distribution:')display(df_spots['band'].value_counts(dropna=True).head(20))print('Band distribution:')# Band and mode distributions, busiest hourpythondist-1codedisplay(top_n(df_spots, 'spotter_call', n))Top spotter calls:')